In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.neural_network import MLPClassifier
#Random Forest
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn import preprocessing

from sklearn.model_selection import StratifiedKFold

import statistics


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
data = pd.read_csv('../input/eestech-challenge-4-local-round-xanthi/dataset.csv')
data.head()
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/eestech-challenge-4-local-round-xanthi/model_scripted.pt
/kaggle/input/eestech-challenge-4-local-round-xanthi/query_model.py
/kaggle/input/eestech-challenge-4-local-round-xanthi/dataset.csv


,poi_0,poi_1,poi_2,poi_3,poi_4,poi_5,poi_6,poi_7,poi_8,poi_9,...,poi_3449,poi_3450,poi_3451,poi_3452,poi_3453,poi_3454,poi_3455,poi_3456,poi_3457,target
0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,16
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,15
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4


In [2]:
data.describe()

,poi_0,poi_1,poi_2,poi_3,poi_4,poi_5,poi_6,poi_7,poi_8,poi_9,...,poi_3449,poi_3450,poi_3451,poi_3452,poi_3453,poi_3454,poi_3455,poi_3456,poi_3457,target
count,7767.000000,7767.000000,7767.000000,7767.000000,7767.000000,7767.000000,7767.000000,7767.000000,7767.000000,7767.000000,...,7767.000000,7767.000000,7767.000000,7767.000000,7767.000000,7767.000000,7767.000000,7767.000000,7767.000000,7767.000000
mean,0.005279,0.007982,0.000772,0.004249,0.017639,0.014935,0.008369,0.000901,0.012489,0.001416,...,0.000386,0.000386,0.000129,0.000129,0.000257,0.000515,0.000644,0.000257,0.000257,6.514484
std,0.072468,0.088993,0.027785,0.065048,0.131643,0.121301,0.091103,0.030009,0.111060,0.037609,...,0.019651,0.019651,0.011347,0.011347,0.016046,0.022689,0.025366,0.016046,0.016046,4.695768
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,17.000000


In [3]:
data.shape

(7767, 3459)

In [4]:
def get_Xy_to_numpy(dataset):
    X = dataset[dataset.columns[:3458]].to_numpy()
    y = dataset[dataset.columns[3458:]].to_numpy().ravel()
    return X, y

In [5]:
def train_sk(train_X, train_Y, test_X, test_Y):
    # initialize classifier
    #classifier = MLPClassifier(hidden_layer_sizes=(32,32,32), random_state=0, max_iter=500)
    classifier = RandomForestClassifier()
    
    # train the classifier
    classifier.fit(train_X,train_Y)

    # make predictions on train/test set
    y_pred_train = classifier.predict(train_X)
    y_pred = classifier.predict(test_X)

    # calculate scores
    accuracy_train = accuracy_score(y_pred_train, train_Y)
    precision_train = precision_score(y_pred_train, train_Y, average='macro')
    recall_train = recall_score(y_pred_train, train_Y, average='macro')
    f1_train = f1_score(y_pred_train, train_Y, average='macro')

    accuracy = accuracy_score(y_pred, test_Y)
    precision = precision_score(y_pred, test_Y, average='macro')
    recall = recall_score(y_pred, test_Y, average='macro')
    f1 = f1_score(y_pred, test_Y, average='macro')

    print(f"[Training set] Accuracy: {accuracy_train:.4f}, Precision: {precision_train:.4f}, Recall: {recall_train:.4f}, F1: {f1_train:.4f}\n",
          f"[Testing set] Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
    return accuracy, precision, recall, f1


In [6]:
def get_statistics(accuracy, precision, recall, f):
    print(f"Stdev:\n\tAcc: {statistics.stdev(accuracy):.4f}\n\tPre: {statistics.stdev(precision):.4f}\n\tRec: {statistics.stdev(recall):.4f}\n\tF1: {statistics.stdev(f):.4f}")
    print(f"Mean Scores:\n\tAcc: {statistics.mean(accuracy):.4f}\n\tPre: {statistics.mean(precision):.4f}\n\tRec: {statistics.mean(recall):.4f}\n\tF1: {statistics.mean(f):.4f}")
    return statistics.mean(f)

In [7]:
X, y = get_Xy_to_numpy(data)
min_max_scaler = preprocessing.MinMaxScaler().fit(X)
X = min_max_scaler.transform(X)

In [8]:
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
global_f1 = dict()
accuracy, precision, recall, f1 = dict(), dict(), dict(), dict()

for i, (train_idx, test_idx) in enumerate(skf.split(X, y)):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    acc, prec, rec, f = train_sk(X_train, y_train, X_test, y_test)
    accuracy[i] = acc
    precision[i] = prec
    recall[i] = rec
    f1[i] = f
mean_f_score = get_statistics(list(accuracy.values()), list(precision.values()), list(recall.values()), list(f1.values()))
global_f1['baseline'] = mean_f_score

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[Training set] Accuracy: 1.0000, Precision: 1.0000, Recall: 1.0000, F1: 1.0000
 [Testing set] Accuracy: 0.8636, Precision: 0.7618, Recall: 0.8421, F1: 0.7726


/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[Training set] Accuracy: 1.0000, Precision: 1.0000, Recall: 1.0000, F1: 1.0000
 [Testing set] Accuracy: 0.8636, Precision: 0.7799, Recall: 0.8451, F1: 0.7779
[Training set] Accuracy: 1.0000, Precision: 1.0000, Recall: 1.0000, F1: 1.0000
 [Testing set] Accuracy: 0.8546, Precision: 0.7985, Recall: 0.9118, F1: 0.8162
[Training set] Accuracy: 1.0000, Precision: 1.0000, Recall: 1.0000, F1: 1.0000
 [Testing set] Accuracy: 0.8597, Precision: 0.7825, Recall: 0.9129, F1: 0.7959
[Training set] Accuracy: 1.0000, Precision: 1.0000, Recall: 1.0000, F1: 1.0000
 [Testing set] Accuracy: 0.8559, Precision: 0.7895, Recall: 0.9086, F1: 0.8109
[Training set] Accuracy: 1.0000, Precision: 1.0000, Recall: 1.0000, F1: 1.0000
 [Testing set] Accuracy: 0.8443, Precision: 0.7462, Recall: 0.8933, F1: 0.7642
[Training set] Accuracy: 1.0000, Precision: 1.0000, Recall: 1.0000, F1: 1.0000
 [Testing set] Accuracy: 0.8662, Precision: 0.7881, Recall: 0.9191, F1: 0.8111
[Training set] Accuracy: 1.0000, Precision: 1.0000, 